In [7]:
import pandas
import helpers
import numpy as np

# Предобработка данных

In [2]:
# Загружаем данные
train = pandas.read_csv('./data/features.csv', index_col='match_id')
test = pandas.read_csv('./data/features_test.csv', index_col='match_id')

# Удаляем признаки, связанные с итогами матча
train.drop(['duration', 
         'tower_status_radiant', 
         'tower_status_dire', 
         'barracks_status_radiant', 
         'barracks_status_dire'
        ], axis=1, inplace=True)

# И разделяем датасет на признаки и целевую переменную
X = train
y = train['radiant_win'].to_frame()
del train['radiant_win']

# Методы для очистки

In [29]:
# Заменяем пропуски на 0
def clean(X):
    return X.fillna(0)

In [28]:
# В данных присутствует 11 категориальных признаков, удаляем их
def clean_category(X):
    X = clean(X)
    del X['lobby_type']
    for n in xrange(1, 6):
        del X['r{}_hero'.format(n)]
        del X['d{}_hero'.format(n)]

    return X

In [13]:
heroes = pandas.read_csv('./data/dictionaries/heroes.csv')
print 'Всего героев в игре:', len(heroes)

Всего героев в игре: 112


In [33]:
# Формируем "мешок слов" по героям
def hero_bag(X):
    X_pick = np.zeros((X.shape[0], len(heroes)))
    for i, match_id in enumerate(X.index):
        for p in xrange(5):
            X_pick[i, X.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
            X_pick[i, X.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

    return pandas.DataFrame(X_pick, index=X.index)

# Очищаем данные и сохраняем для построения модели

In [11]:
helpers.save_clean_data(clean, X, y, test)

In [12]:
helpers.save_clean_data(clean_category, X, y, test, name='clean_category')

In [34]:
helpers.save_clean_data(hero_bag, X, y, test, name='hero_bag')